In [88]:
import pickle
import os


In [277]:
# # input for exp
# base_dir = os.getcwd()
# data_dir = os.path.join(base_dir, 'Input_data')
# input_file = os.path.join(data_dir, 'H2_bravyi_kitaev_2_qubit_experiment_time=2020Sep21-162239536536.pickle')
# with open(input_file, 'rb') as handle:
#     input_data = pickle.load(handle)


In [ ]:
# # ### LOAD DATA


# # seq rot
# base_dir = os.getcwd()
# data_dir = os.path.join(base_dir, 'Data')
# sim_dir = os.path.join(data_dir, 'simulator')
# output_file = os.path.join(sim_dir, 'molecule=H2___n_shots=8190.0___method=seq_rot_VQE___time=2020Sep21-162446389877.pickle')
# with open(output_file, 'rb') as handle:
#     output_data = pickle.load(handle)


# # # LCU
# # base_dir = os.getcwd()
# # data_dir = os.path.join(base_dir, 'Data')
# # sim_dir = os.path.join(data_dir, 'simulator')
# # output_file = os.path.join(sim_dir, 'method=LCU___time=2020Sep21-153600259987.pickle')
# # with open(output_file, 'rb') as handle:
# #     output_data = pickle.load(handle)

# # # standard VQE
# # base_dir = os.getcwd()
# # data_dir = os.path.join(base_dir, 'Data')
# # sim_dir = os.path.join(data_dir, 'simulator')
# # output_file = os.path.join(sim_dir, 'method=standard_VQE___time=2020Sep21-155354364092.pickle')
# # with open(output_file, 'rb') as handle:
# #     output_data = pickle.load(handle)

In [237]:
# ### LOAD DATA


# # seq rot 
base_dir = os.getcwd()
data_dir = os.path.join(base_dir, 'Data')
dev_dir = os.path.join(data_dir, 'device')
Run_1_dir = os.path.join(dev_dir, 'Run_1')
output_file = os.path.join(Run_1_dir, 'molecule=H2___n_shots=8190.0___method=seq_rot_VQE___time=2020Sep22-085203081773.pickle')
with open(output_file, 'rb') as handle:
    output_data = pickle.load(handle)

# # standard VQE
# base_dir = os.getcwd()
# data_dir = os.path.join(base_dir, 'Data')
# sim_dir = os.path.join(data_dir, 'simulator')
# output_file = os.path.join(sim_dir, 'method=standard_VQE___time=2020Sep21-155354364092.pickle')
# with open(output_file, 'rb') as handle:
#     output_data = pickle.load(handle)

In [107]:
list(output_data.keys())

['start_time',
 'end_time',
 'method',
 'n_shots',
 'I_term',
 'q_circuit_list',
 'SIM_transpiled_q_circuit_list',
 'experiment_dict',
 'SIM_count_list_raw',
 'jobID',
 'EXP_transpiled_q_circuit_list',
 'EXP_backend',
 'EXP_count_list_raw',
 'standard_measurement_filter']

In [84]:
output_data

{'start_time': datetime.datetime(2020, 9, 21, 16, 24, 46, 358109),
 'end_time': datetime.datetime(2020, 9, 21, 16, 24, 46, 358602),
 'method': 'seq_rot_VQE',
 'n_shots': 8190.0,
 'I_term': (0.2460355896585992+0j),
 'q_circuit_list': [<qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7f109ef6fe90>,
 'SIM_transpiled_q_circuit_list': [<qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7f109eef9ed0>,
 'experiment_dict': [{'circuit': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7f109ef6fe90>,
   'coeff': (0.5731061703432151+0j),
   'qubitOp': (0.5731061703432151+0j) [Z0 Z1]},
  {'circuit': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7f109ee77cd0>,
   'gamma_l': (0.455956044621043+0j),
   'Ps': 1 [Z1]},
  {'circuit': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7f109ee7f490>,
   'gamma_l': (0.35459658228639496+0j),
   'Ps': 1 [Z0]}],
 'SIM_count_list_raw': [{'01': 8069, '10': 121},
  {'01': 3, '10': 8187},
  {'01': 8185, '10': 5}]}

In [238]:
from quchem_ibm.Qiskit_Chemistry import *

def Calc_Energy_seq_rot(experimental_count_list, simulated_count_list, list_experiment_dicts, I_term, standard_meas_filter):
    
    E_raw_list=[I_term]
    E_sim_list=[I_term]
    E_mit_list=[I_term]
    
    for index, exp_dict in enumerate(list_experiment_dicts):
        
        exp_counts_dict=experimental_count_list[index]
        simulated_counts=simulated_count_list[index]
        meas_mit_counts= standard_meas_filter.apply(exp_counts_dict)
        if 'Ps' in exp_dict.keys():
            Ps=exp_dict['Ps']
            gamma_l = exp_dict['gamma_l']
            
            # raw
            exp_val = calc_exp_pauliword(exp_counts_dict, Ps)
            E_raw_list.append(exp_val*gamma_l)
            
            # sim
            exp_val_sim = calc_exp_pauliword(simulated_counts, Ps)
            E_sim_list.append(exp_val_sim*gamma_l)
            
            # mitigated
            exp_val_mit = calc_exp_pauliword(meas_mit_counts,Ps)
            E_mit_list.append(exp_val_mit*gamma_l)
        else:
            qubitOp=exp_dict['qubitOp']
            coeff = exp_dict['coeff']
            
            # raw
            exp_val = calc_exp_pauliword(exp_counts_dict, qubitOp)
            E_raw_list.append(exp_val*coeff)

            # sim
            exp_val_sim = calc_exp_pauliword(simulated_counts, qubitOp)
            E_sim_list.append(exp_val_sim*coeff)
            
            # mitigated
            exp_val_mit = calc_exp_pauliword(meas_mit_counts,qubitOp)
            E_mit_list.append(exp_val_mit*coeff)
    return sum(E_raw_list), sum(E_sim_list), sum(E_mit_list)

In [239]:
Calc_Energy_seq_rot(output_data['EXP_count_list_raw'], 
                    output_data['SIM_count_list_raw'],
                    output_data['experiment_dict'], 
                    output_data['I_term'], 
                    output_data['standard_measurement_filter'])

((-0.644076125115526+0j), (-1.1371654927266652+0j), (-1.0674952487505802+0j))

In [108]:
# LCU
base_dir = os.getcwd()
data_dir = os.path.join(base_dir, 'Data')
dev_dir = os.path.join(data_dir, 'device')
Run_1_dir = os.path.join(dev_dir, 'Run_1')
output_file = os.path.join(Run_1_dir, 'molecule=H2___n_shots=8190.0___method=LCU___time=2020Sep22-094030135037.pickle')
with open(output_file, 'rb') as handle:
    output_data = pickle.load(handle)


In [234]:
list(output_data.keys())

['start_time',
 'end_time',
 'method',
 'n_shots',
 'I_term',
 'q_circuit_list',
 'SIM_transpiled_q_circuit_list',
 'experiment_dict',
 'SIM_count_list_raw',
 'jobID',
 'EXP_transpiled_q_circuit_list',
 'EXP_backend',
 'EXP_count_list_raw',
 'standard_measurement_filter',
 'SIM_memory_raw',
 'EXP_memory_raw',
 'LCU_measurement_filters']

In [232]:
def Calc_Energy_LCU(experimental_meas_list, simulated_meas_list, list_experiment_dicts, I_term,
                    LCU_measurement_filter, standard_meas_filter):
    E_raw_list = [I_term]
    E_sim_list = [I_term]
    E_mit_list = [I_term]

    for index, exp_dict in enumerate(list_experiment_dicts):

        exp_measurements = experimental_meas_list[index]
        simulated_measurements = simulated_meas_list[index]

        if 'Pn' in exp_dict.keys():
            Pn = exp_dict['Pn']
            gamma_l = exp_dict['gamma_l']
            N_ancilla = exp_dict['N_ancilla']

            # raw
            post_selected_dict_raw = Get_post_selection_counts_LCU(exp_measurements, N_ancilla)
            exp_val = calc_exp_pauliword(post_selected_dict_raw, Pn)
            E_raw_list.append(exp_val * gamma_l)

            # sim
            post_selected_dict_sim = Get_post_selection_counts_LCU(simulated_measurements, N_ancilla)
            exp_val_sim = calc_exp_pauliword(post_selected_dict_sim, Pn)
            E_sim_list.append(exp_val_sim * gamma_l)

            # mitigated
            raw_counts_dict = Get_post_selection_counts_LCU(exp_measurements, 0)
            mit_counts_dict = LCU_measurement_filter.apply(raw_counts_dict)
            
            post_selected_dict_mit = Get_post_selection_counts_DICT_LCU(mit_counts_dict, N_ancilla)
            exp_val_mit = calc_exp_pauliword(post_selected_dict_mit, Pn)
            E_mit_list.append(exp_val_mit * gamma_l)

        else:
            qubitOp = exp_dict['qubitOp']
            coeff = exp_dict['coeff']

            # raw
            post_selected_dict_raw = Get_post_selection_counts_LCU(exp_measurements, 0)
            exp_val = calc_exp_pauliword(post_selected_dict_raw, qubitOp)
            E_raw_list.append(exp_val * coeff)

            # sim
            post_selected_dict_sim = Get_post_selection_counts_LCU(simulated_measurements, 0)
            exp_val_sim = calc_exp_pauliword(post_selected_dict_sim, qubitOp)
            E_sim_list.append(exp_val_sim * coeff)

            # mitigated
            post_selected_dict_mit = standard_meas_filter.apply(post_selected_dict_raw)
            exp_val_mit = calc_exp_pauliword(post_selected_dict_mit, qubitOp)
            E_mit_list.append(exp_val_mit * coeff)
            

    return sum(E_raw_list), sum(E_sim_list), sum(E_mit_list)

In [156]:
# list(output_data.keys())

In [233]:
Calc_Energy_LCU(output_data['EXP_memory_raw'],
                output_data['SIM_memory_raw'],
                output_data['experiment_dict'],
                output_data['I_term'],
                output_data['LCU_measurement_filters'][0]['filter'],
                output_data['standard_measurement_filter'])

((-0.6169571941620106+0j), (-1.137515851678555+0j), (-1.0456976549546295+0j))

In [242]:
# STANDARD RESULTSAbs

base_dir = os.getcwd()
data_dir = os.path.join(base_dir, 'Data')
dev_dir = os.path.join(data_dir, 'device')
Run_1_dir = os.path.join(dev_dir, 'Run_1')
output_file = os.path.join(Run_1_dir, 'molecule=H2___n_shots=4914.0___method=standard_VQE___time=2020Sep22-092646468985.pickle')
with open(output_file, 'rb') as handle:
    output_data = pickle.load(handle)


In [243]:
from quchem_ibm.Qiskit_Chemistry import *

def Calc_Energy_standard(experimental_count_list, simulated_count_list, list_experiment_dicts, I_term, standard_meas_filter):
    
    E_raw_list=[I_term]
    E_sim_list=[I_term]
    E_mit_list=[I_term]
    
    for index, exp_dict in enumerate(list_experiment_dicts):
        
        exp_counts_dict=experimental_count_list[index]
        simulated_counts=simulated_count_list[index]
        meas_mit_counts= standard_meas_filter.apply(exp_counts_dict)

        
        qubitOp=exp_dict['qubitOp']
        coeff = exp_dict['coeff']

        # raw
        exp_val = calc_exp_pauliword(exp_counts_dict, qubitOp)
        E_raw_list.append(exp_val*coeff)

        # sim
        exp_val_sim = calc_exp_pauliword(simulated_counts, qubitOp)
        E_sim_list.append(exp_val_sim*coeff)

        # mitigated
        exp_val_mit = calc_exp_pauliword(meas_mit_counts,qubitOp)
        E_mit_list.append(exp_val_mit*coeff)
        
    return sum(E_raw_list), sum(E_sim_list), sum(E_mit_list)

In [245]:
Calc_Energy_standard(output_data['EXP_count_list_raw'], 
                    output_data['SIM_count_list_raw'],
                    output_data['experiment_dict'], 
                    output_data['I_term'], 
                    output_data['standard_measurement_filter'])

((-0.8678025427223981+0j), (-1.1363507128235555+0j), (-1.0503310872627942+0j))

In [247]:
# input for exp
base_dir = os.getcwd()
data_dir = os.path.join(base_dir, 'Input_data')
input_file = os.path.join(data_dir, 'LiH_bravyi_kitaev_12_qubit_experiment_time=2020Sep21-132537266776.pickle')
with open(input_file, 'rb') as handle:
    input_data = pickle.load(handle)
list(input_data.keys())

['Hamiltonian',
 'anti_commuting_sets',
 'geometry',
 'basis_set',
 'transformation',
 'Graph_colouring_strategy',
 'fci_energy',
 'standard_VQE_circuits',
 'standard_I_term',
 'Seq_Rot_VQE_circuits',
 'S_indices_dict',
 'Seq_Rot_I_term',
 'LCU_VQE_circuits',
 'LCU_I_term',
 'N_index_dict',
 'n_system_qubits',
 'ground_state']

In [249]:
len(input_data['LCU_VQE_circuits'])

102

In [250]:
len(input_data['standard_VQE_circuits'])

630

In [275]:
print(10710/630)
print(10710/102)

17.0
105.0


In [276]:
10710/2

5355.0